In [1]:
import ipysheet
import ipywidgets as widgets
import traitlets
from IPython.display import display, clear_output

from src.post_processing import PathWrangler, Path, Enzyme
from src.widgets import sheet_from_dataclass

In [2]:
pw = PathWrangler(
    path_filepath='../artifacts/processed_expansions/found_paths.json',
    pr_filepath='../artifacts/processed_expansions/predicted_reactions.json',
    kr_filepath='../artifacts/processed_expansions/known_reactions.json',
)

In [3]:
# UI widgets and layout

# input

starter_options, target_options = map(sorted, zip(*pw.starter_targets))
target_options = sorted(set(target_options))
evidence_options = [(elt.value, elt.name) for elt in pw.enzyme_existence]
sort_options = [
    ("Mean RCMCS", 'mean_rcmcs'),
    ("Min RCMCS", 'min_rcmcs'),
    ("Max-min driving force", 'mdf')
]
kwargs_input_selector = dict(
    disabled=False,
    continuous_update=False,
    rows=len(starter_options),
    layout=widgets.Layout(
        width='auto',
        flex='1 1 auto',
    ),
    style=dict(
        description_width='100px',
    ),
)

input_selectors = widgets.HBox(
    children=[
        starter_selector := widgets.SelectMultiple(
            description='Starters:',
            options=starter_options,
            value=starter_options[:1],
            **kwargs_input_selector,
        ),
        target_selector := widgets.SelectMultiple(
            description='Targets:',
            options=target_options,
            value=target_options,
            **kwargs_input_selector,
        ),
        evidence_selector := widgets.SelectMultiple(
            description='Enzyme LOE:',
            options=evidence_options,
            value=[pw.enzyme_existence.PROTEIN.name],
            **kwargs_input_selector,
        ),
        sort_by_radio_buttons := widgets.Select(
            description='Sort paths by:',
            options=sort_options,
            value='mean_rcmcs',
            **kwargs_input_selector,
        ),
    ],
    layout=widgets.Layout(
        flex_flow='row wrap',
    ),
)

# output 

kwargs_output_selector = {
    **kwargs_input_selector,
    'layout': widgets.Layout(
        width='auto',
        flex='0 1 auto',
    ),
}

output_paths = widgets.VBox(
    children=[
        paths_label := widgets.Label(
            value='Awaiting path exploration',
        ),
        paths_selector := widgets.SelectMultiple(
            description='Paths:',
            **kwargs_output_selector,
        ),
        paths_viewer := widgets.Output(),
    ],
)

# UI overall

ui = widgets.VBox(
    children=[
    input_selectors,
    output_paths,
    ],
    layout=widgets.Layout(
        width='80%',
        justify_content='flex-start',
    ),
)

In [4]:
# custom widget builders

def build_uniprot_link(uniprot_id: str) -> widgets.HTML:
    return widgets.HTML(
        f'<a href="https://www.uniprot.org/uniprotkb/{uniprot_id}/entry" target="_blank" rel="noopener noreferrer">{uniprot_id}</a>'
    )


def build_enzymes(enzymes: list[Enzyme]) -> ipysheet.Sheet:
    return sheet_from_dataclass(
        data=enzymes,
        field_transformers=dict(
            uniprot_id=lambda dc, field_name: dict(
                value=build_uniprot_link(uniprot_id=getattr(dc, field_name)),
                type='widget',
            )
        )
    )


def display_predicted_reaction(rxn_step, img):
    html = widgets.HTML(f'<b><u>Step #{rxn_step + 1}</u></b>')
    svg = widgets.Image.from_file(img)
    svg.width = 650
    return widgets.VBox([html, svg])


def display_analogue(img, rcmcs):
    html = widgets.HTML(f'<b><u>{rcmcs * 100:.2f}% similar to predicted reaction</u></b>')
    svg = widgets.Image.from_file(img)
    svg.width = 600
    return widgets.VBox([html, svg])


def widget_analogues_enzymes(krs, rcmcses):
    kr_elts = []
    enzyme_elts = []
    for kr, rcmcs in zip(krs, rcmcses):
        kr_elts.append(display_analogue(kr.image, rcmcs))
        enzyme_elts.append(build_enzymes(kr.enzymes))

    kr_selector = widgets.Dropdown(
        options=[(i + 1, i) for i in range(len(kr_elts))],
        value=0,
        description="Analogue: "
    )
    kr_stack = widgets.Stack(kr_elts, selected_index=0)
    kr_sel_disp = widgets.VBox([kr_selector, kr_stack])
    enzyme_stack = widgets.Stack(enzyme_elts, selected_index=0)
    _link_kr = widgets.jslink((kr_selector, 'index'), (kr_stack, 'selected_index'))
    _link_enz = widgets.jslink((kr_selector, 'index'), (enzyme_stack, 'selected_index'))

    tab_titles = ['Known Analogues', 'Enzymes']
    tab_children = [kr_sel_disp, enzyme_stack]

    return widgets.Tab(
        children=tab_children,
        titles=tab_titles,
        layout=widgets.Layout(width="950px")
    )


def widget_path_view(path: Path, k: int = 1000):
    header = widgets.HTML(f"""
    <h3>{len(path.reactions)}-step path from {path.starter.upper()} to {path.target.upper()}<br>
    Max-min driving force = {path.mdf:.2f} kJ/mol<br>
    </h3>
    """)
    rows = [header]
    for i, rxn in enumerate(path.reactions):
        pr_elt = display_predicted_reaction(i, rxn.image)
        kr_elt = widget_analogues_enzymes(
            krs=rxn.top_analogues(k=k),
            rcmcses=rxn.top_rcmcs(k=k),
        )
        row = widgets.HBox([pr_elt, kr_elt], layout=widgets.Layout(border='1px solid black'))
        rows.append(row)
    return widgets.VBox(rows)

In [5]:
# event handlers and wiring

# on input change :: fetch paths & update path selector

def update_paths_selector(_change: traitlets.Bunch):
    path_objs = pw.get_paths(
        starters=starter_selector.value,
        targets=target_selector.value,
        filter_by_enzymes={'existence': evidence_selector.value},
        sort_by=sort_by_radio_buttons.value,
    )
    paths_selector.disabled = not path_objs
    options = [(idx + 1, path) for idx, path in enumerate(path_objs)]
    paths_selector.options = options
    if options:
        # avoid setting `value` when the options are empty
        # otherwise, traitlet validations go berserk
        paths_selector.value = [options[0][1]]


for input_selector in input_selectors.children:
    input_selector.observe(update_paths_selector, names=['value'])


# on paths selection change :: update paths label & render the paths viewer

def update_paths_label(change: traitlets.Bunch):
    paths_label.value = f'Fetched {len(change.owner.options)} paths'


def render_paths(change: traitlets.Bunch):
    with paths_viewer:
        clear_output()
        display(
            widgets.VBox([widget_path_view(path) for path in change.new])
        )


paths_selector.observe(update_paths_label, names=['value'])
paths_selector.observe(render_paths, names=['value'])

In [6]:
# display UI & kick off a selection
display(ui)
starter_selector.value = starter_options[:2]